# Laboratorio 2 - Visión por computadora

- Nelson García
- Joaquín Puente 
- Diego Linares

## Task 3 

Una fábrica textil necesita detectar rasgaduras en telas de mezclilla (denim) automáticamente. El problema
es que la tela tiene una textura natural fuerte (patrón repetitivo) que confunde a los detectores de bordes
simples (Canny), detectando el tejido como si fuera un defecto. Por ello se le pide que usted diseñe un
pipeline híbrido que combine Fourier y Morfología para aislar solamente la rasgadura. Para ello comienza
por probar su solución en una imagen que tiene a mano. Con esto en mente, realice:

In [1]:
# Importar librerías:

import argparse
import numpy as np
import cv2
import matplotlib.pyplot as plt

1. Utilice Fourier para analizar la textura repetitiva de la tela. Diseñe un filtro que elimine las
frecuencias altas/repetitivas del tejido, dejando una imagen "suavizada" donde solo resalte la
anomalía (la rasgadura) y la iluminación global. (Supresión de Textura)
a. Hint: ¿Qué pasa si eliminamos las frecuencias altas periféricas o específicas?

In [12]:
IMG_NAME = "pantalon.jpg"

In [13]:
LOWPASS_SIGMA = 35       # más alto = más suavizado (menos textura)
NOTCH_PEAKS = 8          # cuántos picos de textura apagar
NOTCH_RADIUS = 10        # tamaño del "agujero" alrededor del pico


In [14]:
# --- Cargar imagen en gris ---
img = cv2.imread(IMG_NAME, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"No pude leer {IMG_NAME}. ¿Está en el mismo folder?")

h, w = img.shape
cy, cx = h // 2, w // 2

In [15]:
# --- FFT (pasar a frecuencia) ---
F = np.fft.fft2(img.astype(np.float32))
Fshift = np.fft.fftshift(F)

In [16]:
print(Fshift)

[[  2215.     -1.3642421e-12j   2380.5767 -5.1049067e+03j
   -2070.8687 -2.8526047e+03j ...   -290.23743+3.8475708e+03j
   -2070.8687 +2.8526047e+03j   2380.5767 +5.1049067e+03j]
 [-12283.457  +2.4655271e+03j   4210.351  -2.3299655e+02j
   -3281.7292 +1.9562247e+03j ...  -4639.519  +2.4101008e+03j
   -3629.9941 +3.6270663e+02j    932.88824+1.0001198e+04j]
 [ -6265.261  -3.6522998e+03j   5856.548  +1.7418792e+03j
    1861.155  +8.5919183e+02j ...  16864.727  +5.3294824e+03j
  -17490.84   -9.6695527e+03j  -1783.3756 +1.9722166e+03j]
 ...
 [  2317.0847 -3.7004303e+01j  -3534.1816 -1.1644892e+04j
    9348.76   +7.7026958e+03j ...   -239.8946 +3.5095205e+03j
   -5474.5713 +7.4914976e+03j    424.82217-3.1360950e+03j]
 [ -6265.261  +3.6522998e+03j  -1783.3756 -1.9722166e+03j
  -17490.84   +9.6695527e+03j ...   5994.935  -3.6947844e+03j
    1861.155  -8.5919183e+02j   5856.548  -1.7418792e+03j]
 [-12283.457  -2.4655271e+03j    932.88824-1.0001198e+04j
   -3629.9941 -3.6270663e+02j ... -10605.7

In [17]:
# Magnitud (para visualizar el espectro y encontrar picos)
mag = np.log1p(np.abs(Fshift))

In [18]:
# --- Filtro 1: Low-pass gaussiano (quita altas frecuencias periféricas) ---
yy, xx = np.mgrid[0:h, 0:w]
dist2 = (yy - cy) ** 2 + (xx - cx) ** 2
lowpass = np.exp(-dist2 / (2.0 * (LOWPASS_SIGMA ** 2))).astype(np.float32)

# --- Filtro 2: Notch (apaga picos fuertes repetitivos del tejido) ---
mag2 = mag.copy()

In [ ]:
# Quitar la zona central para no escoger "iluminación global" como pico
center_block = 25
mag2[cy - center_block:cy + center_block, cx - center_block:cx + center_block] = 0


In [ ]:
# Elegir los picos más fuertes (aprox. patrones repetitivos)
flat = mag2.ravel()
idxs = np.argpartition(flat, -NOTCH_PEAKS)[-NOTCH_PEAKS:]
coords = np.column_stack(np.unravel_index(idxs, mag2.shape))


In [ ]:
notch = np.ones((h, w), dtype=np.float32)
for y, x in coords:
    # Apaga un disco alrededor del pico y su simétrico (porque FFT de imagen real es simétrica)
    cv2.circle(notch, (int(x), int(y)), NOTCH_RADIUS, 0, -1)
    y2, x2 = int(2 * cy - y), int(2 * cx - x)
    if 0 <= y2 < h and 0 <= x2 < w:
        cv2.circle(notch, (x2, y2), NOTCH_RADIUS, 0, -1)

In [ ]:
# --- Máscara final en frecuencia ---
freq_mask = lowpass * notch

# --- Aplicar filtro en frecuencia ---
F_filt = Fshift * freq_mask

# --- IFFT (volver al dominio espacial) ---
img_back = np.fft.ifft2(np.fft.ifftshift(F_filt))
img_back = np.real(img_back)

# Normalizar para visualizar bien
smooth = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)


In [ ]:
# --- Mostrar etapas ---
plt.figure()
plt.title("Original (gris)")
plt.imshow(img, cmap="gray")
plt.axis("off")



In [ ]:
plt.figure()
plt.title("FFT Magnitud (log)")
plt.imshow(mag, cmap="gray")
plt.axis("off")

In [ ]:
plt.figure()
plt.title("Máscara Low-pass")
plt.imshow(lowpass, cmap="gray")
plt.axis("off")

In [ ]:
plt.figure()
plt.title("Máscara Notch (picos apagados)")
plt.imshow(notch, cmap="gray")
plt.axis("off")

In [ ]:
plt.figure()
plt.title("Máscara final (Low-pass * Notch)")
plt.imshow(freq_mask, cmap="gray")
plt.axis("off")


In [ ]:
plt.figure()
plt.title("Resultado IFFT (textura suprimida)")
plt.imshow(smooth, cmap="gray")
plt.axis("off")

In [ ]:
plt.show()